In [1]:
# Create an area map that contains a grid of ?'s and the position
function make_area_map(n)
    grid = fill('?', n, n)
    ctr = n ÷ 2
    pos = [ctr, ctr]
    (pos = pos, grid = grid)
end

# Visualize the area map.
# Show current position with X.
# Display # instead of ? when `show_wall` is true.
function visualize(amap; show_walls = true)
    for y in 1:size(amap.grid,1)
        for x in 1:size(amap.grid, 2)
            let c = amap.grid[y,x]
                if x == amap.pos[1] && y == amap.pos[2]
                    print('X')
                else
                    print(c == '?' && show_walls ? '#' : c)
                end
            end
        end
        println()
    end
end

# Move one step in a specific direction - 'N', 'S', 'E', or 'W'
# Mark the crossover position with either '-' or '|'
# Update current position in the map
function move!(amap, dir::Char)
    x, y = amap.pos
    let grid = amap.grid
        grid[y, x] = '.'   # mark current pos
        if dir == 'N'
            grid[y-1, x] = '-'
            amap.pos[2] = y-2
        elseif dir == 'S'
            grid[y+1, x] = '-'
            amap.pos[2] = y+2
        elseif dir == 'W'
            grid[y, x-1] = '|'
            amap.pos[1] = x-2
        else  # East
            grid[y, x+1] = '|'
            amap.pos[1] = x+2
        end
    end
end

# Teleport to another position
function teleport!(amap, pos)
    x, y = amap.pos
    amap.grid[y,x] = '.'  # mark current pos
    amap.pos[:] = pos[:]
end

# Move according to the regex instruction
# The starting position is passed as `origx` and `origy`.
# Keep track of the current `level` as it's a recursive function.
function move!(amap, guide::String, origx, origy, level = 0; debug = false)
    debug && println("==> level=", level, " guide=", guide, " orig=(", origx, ",", origy, ")")
    i = 1
    while i <= length(guide)
        c = guide[i]
        if c == '('   # recursion starts here; advance to ')'
            amap, state = move!(amap, guide[i+1:end], amap.pos[1], amap.pos[2], level + 1)
            i += state
        elseif c == ')'  # returns to caller 
            break
        elseif c == '|'  # teleport back to the position when recursion began
            debug && println("==> level=", level, " teleporting to (", origx, ",", origy, ")")
            teleport!(amap, [origx, origy])
        else  # move one step when processing normal direction char
            move!(amap, c)
        end
        i += 1
    end
    (amap, i)
end

# the furthest path can be found from enumerating the next
# possible step and from that next step recursively do the 
# same calculation.  Need to make sure that a room that
# has been visited before is not considered again to avoid
# infinite loop.
function furthest_path(amap, x, y, distance = 0, visited = [], level = 0)
    next_steps = possible_next_steps(amap, x, y, visited, level)
    if length(next_steps) > 0
        maximum(furthest_path(amap, s.x, s.y, distance + 1, [visited; s], level + 1) 
            for s ∈ next_steps)
    else
#         println(level, ": distance=", distance, ", x=", x, ", y=", y, ", visited=", visited)
        distance
    end
end
    
function possible_next_steps(amap, x, y, visited, level)
    next_steps = []
    let pos = (x = x, y = y - 2), pass = (x = x, y = y - 1)
        amap.grid[pos.y, pos.x] == '.' && amap.grid[pass.y, pass.x] == '-' && 
        !in(pos, visited) && push!(next_steps, pos)
    end
    let pos = (x = x, y = y + 2), pass = (x = x, y = y + 1)
        amap.grid[pos.y, pos.x] == '.' && amap.grid[pass.y, pass.x] == '-' && 
        !in(pos, visited) && push!(next_steps, pos)
    end
    let pos = (x = x - 2, y = y), pass = (x = x - 1, y = y)
        amap.grid[pos.y, pos.x] == '.' && amap.grid[pass.y, pass.x] == '|' && 
        !in(pos, visited) && push!(next_steps, pos)
    end
    let pos = (x = x + 2, y = y), pass = (x = x + 1, y = y)
        amap.grid[pos.y, pos.x] == '.' && amap.grid[pass.y, pass.x] == '|' && 
        !in(pos, visited) && push!(next_steps, pos)
    end
#     println("At (", x, ",", y, ") possible next steps=", next_steps)
    next_steps
end


function testme(s, n; display = true)
    amap = make_area_map(n)
    home = copy(amap.pos)
    move!(amap, s, home[1], home[2])
    teleport!(amap, home)
    display && visualize(amap)
    furthest_path(amap, home[1], home[2])
end

testme (generic function with 1 method)

In [2]:
testme("ENWWW(NEEE|SSE(EE|N))", 15)

###############
###############
##.|.|.|.######
##-############
##.|.|.|.######
##-#####-######
##.#.#X|.######
##-#-##########
##.|.|.|.######
###############
###############
###############
###############
###############
###############


10

In [3]:
testme("ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))", 20)

####################
####################
####################
###.|.|.|.|.|.######
###-#####-###-######
###.#.|.#.#.#.######
###-#-###-#-#-######
###.#.#.|.#.|.######
###-#-#-#####-######
###.#.#.#X|.#.######
###-#-#-###-#-######
###.|.#.|.#.#.######
#####-#-###-#-######
###.|.#.|.|.#.######
####################
####################
####################
####################
####################
####################


23

In [4]:
testme("WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))", 30)

##############################
##############################
##############################
##############################
##############################
##############################
##############################
##############################
########.|.|.|.#.|.|.#########
########-###-###-#-#-#########
########.|.#.|.|.#.#.#########
########-#########-#-#########
########.#.|.|.|.|.#.#########
########-#-#########-#########
########.#.#.|X#.|.#.#########
##########-#-###-#-#-#########
########.|.#.#.|.#.|.#########
########-###-#####-###########
########.|.#.|.|.#.#.#########
########-#-#####-#-#-#########
########.#.|.|.|.#.|.#########
##############################
##############################
##############################
##############################
##############################
##############################
##############################
##############################
##############################


31

In [5]:
testme(readline("input20.txt")[2:end-1], 1000, display = false)

3966

In [7]:
# part 2 -- return cells that are reachable within a distance 
function reachable_rooms(amap, x, y, within, distance = 0, visited = [], level = 0)
    distance >= within && return visited   # stop traversing
    next_steps = possible_next_steps(amap, x, y, visited, level)
    if length(next_steps) > 0
        visited_array = [reachable_rooms(amap, s.x, s.y, within, 
                distance + 1, [visited; s], level + 1) 
            for s ∈ next_steps]
        union(visited_array...)
    else
        visited
    end
end

function total_rooms(amap)
    count(x -> x == '.', amap.grid)
end

function testme2(s, n, within; display = true)
    amap = make_area_map(n)
    home = copy(amap.pos)
    move!(amap, s, home[1], home[2])
    teleport!(amap, home)
    display && visualize(amap)
    @show reachable = length(reachable_rooms(amap, home[1], home[2], within))
    @show rooms = total_rooms(amap)
    rooms - reachable
end

testme2("WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))", 30, 12)

##############################
##############################
##############################
##############################
##############################
##############################
##############################
##############################
########.|.|.|.#.|.|.#########
########-###-###-#-#-#########
########.|.#.|.|.#.#.#########
########-#########-#-#########
########.#.|.|.|.|.#.#########
########-#-#########-#########
########.#.#.|X#.|.#.#########
##########-#-###-#-#-#########
########.|.#.#.|.#.|.#########
########-###-#####-###########
########.|.#.|.|.#.#.#########
########-#-#####-#-#-#########
########.#.|.|.|.#.|.#########
##############################
##############################
##############################
##############################
##############################
##############################
##############################
##############################
##############################
reachable = length(reachable_rooms(amap, home[1], home[2], within)) = 

35

In [8]:
testme2(readline("input20.txt")[2:end-1], 1000, 999, display = false)

reachable = length(reachable_rooms(amap, home[1], home[2], within)) = 1827
rooms = total_rooms(amap) = 10000


8173